In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from get_book import *
import importlib
from fictiondb import *

In [23]:
# Create url list (for 12 pages)
listopia_base_url="https://www.goodreads.com/list/show/4271.I_Only_Watched_the_Movie_"
listopia_url_list=[listopia_base_url]
for i in range(2,13):
    url = listopia_base_url + '?page=' + str(i)
    listopia_url_list.append(url)

In [7]:
# Create soup_list (12 soup for 12 pages)
def get_soup(url):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    return soup

# listopia_soup_list=[]
# for url in listopia_url_list:
#     listopia_soup_list.append(get_soup(url))

In [28]:
# listopia_url_list

In [33]:
book_title_list=[]
book_url_list=[]
base_book_url="https://www.goodreads.com/"

for page_url in listopia_url_list:
    soup = get_soup(page_url)
    table= soup.find('table', class_="tableList js-dataTooltip")
    rows = [row for row in table.find_all('tr')]

    for i in range(100):
        book_title = rows[i].find('span', attrs={'itemprop':'name','role':'heading'}).text.split('(')[0].strip(' ')
        book_title_list.append(book_title)
        book_url = rows[i].find_all('a')[0].get('href')
        book_url_list.append(base_book_url+book_url)

IndexError: list index out of range

In [34]:
len(book_url_list)

1122

In [35]:
len(book_title_list)

1122

In [44]:
# Create DataFrame of book+link
book_link_df = pd.DataFrame(list(zip(book_title_list,book_url_list)),\
            columns=['book','link_b'])

In [68]:
# book_link_df

In [57]:
# Load movie_df and complete_data_cleaned to find movie still needs book
movie_df = pd.read_pickle('../dump/movie_data')
all_df = pd.read_pickle('../dump/complete_data_cleaned')

In [58]:
movie_list = list(movie_df.movie_title)
all_movie_list = list(all_df.movie_title)

In [59]:
len(movie_list), len(all_movie_list)

(2000, 581)

In [182]:
'Mary Poppins' in missing_book

True

In [183]:
'Mary Poppins' in all_movie_list

False

In [172]:
missing_book = set(movie_list).difference(set(all_movie_list))
len(missing_book)

1347

In [175]:
missing_book

{'Body Snatchers',
 "What's Love Got to Do with It",
 'A Love Song for Bobby Long',
 'The Wrong Man',
 'Le divorce',
 'The Shootist',
 'The Sound and the Fury',
 'Keys to Tulsa',
 '52 Pick-Up',
 'The Phantom Tollbooth',
 'The Teahouse of the August Moon',
 'The American',
 'Bloodline',
 'Muppet Treasure Island',
 'Inside Moves',
 'Kiss of the Spider Woman',
 'Awakenings',
 'Mr. Majestyk',
 'The Celestine Prophecy',
 'Lord of the Flies',
 'Mad Love',
 'The Letter',
 'Peeper',
 'Artemis Fowl',
 'The Trouble with Angels',
 'The Harrad Experiment',
 'The Stunt Man',
 'Cinderella Liberty',
 'South Central',
 'Just Tell Me What You Want',
 'Slumdog Millionaire',
 'Of Unknown Origin',
 'The Dreamers',
 'Chitty Chitty Bang Bang',
 'House Calls',
 'The Harder They Fall',
 'My Cousin Rachel',
 'Shooter',
 'Mercury Rising',
 'Stay Hungry',
 'Mariette in Ecstasy',
 'The Lost Weekend',
 "Charlie Wilson's War",
 'The Lords of Discipline',
 'The Killer Elite',
 'Topper',
 'Small Apartments',
 'The Ci

In [174]:
overlap_book = set(missing_book).intersection(set(book_title_list))
overlap_book_list = list(overlap_book)
len(overlap_book_list)

114

In [178]:
add_book_df = book_link_df[book_link_df.book.isin(overlap_book_list)]
overlap_book_url_list = list(add_book_df.link_b)
add_book_df

,book,link_b
4,Mary Poppins,https://www.goodreads.com//book/show/152380.Ma...
18,Bambi,https://www.goodreads.com//book/show/739840.Bambi
35,The Phantom of the Opera,https://www.goodreads.com//book/show/480204.Th...
41,The War of the Worlds,https://www.goodreads.com//book/show/8909.The_...
62,Pride and Prejudice,https://www.goodreads.com//book/show/1885.Prid...
...,...,...
1100,The Missing,https://www.goodreads.com//book/show/1054596.T...
1105,The Witches,https://www.goodreads.com//book/show/10594138-...
1118,The Private Lives of Pippa Lee,https://www.goodreads.com//book/show/2116927.T...
1119,The Handmaid's Tale,https://www.goodreads.com//book/show/38447.The...


In [184]:
add_book_df[add_book_df.book == 'The Godfather']

,book,link_b


In [185]:
# Define function to get book data from the book page
# Try fixing missing value problem
# Try fixing first published year problem (if there's only first published year)


def get_bookdata(book): # book = soup in book_soup_list
    
    headers=['book_title', 'author', 'rating_value', 'rating_count', 'review_count', 'page', 'year']
    
    # Assign default value
    title, author, rating_value, rating_count, review_count, page, year = \
    np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    # Find book title, author, rating_value, rating_count, review_count, page
    title = book.find('h1').text.strip('\n').lstrip()
    author = book.find('a', class_="authorName").text
    rating_value = float(book.find('span', attrs={'itemprop':'ratingValue'}).text.strip('\n').lstrip())
    rating_count = int(book.find('meta', attrs={'itemprop':'ratingCount'}).get('content'))
    review_count = int(book.find('meta', attrs={'itemprop':'reviewCount'}).get('content'))
    if book.find('span', attrs={'itemprop':'numberOfPages'}) is not None:
        page = int(book.find('span', attrs={'itemprop':'numberOfPages'}).text.strip(' pages'))

    # Find year first published
    for div in book.find_all('div', attrs={'id':'details'}):
        for row in div.find_all('nobr', class_='greyText'):
            year = row.text
            year = int(''.join(i for i in year if i.isdigit())[-4:])
            
    
    
    # Find earliest year
    if div.find_all('div', class_='row') != []:
        string = div.find_all('div', class_='row')[-1].text
        str_clean = string.replace('(',' ').replace(')',' ').split()
        year_list = [int(s) for s in str_clean if s.isdigit()]
        if year_list != []:
            year = min(year_list)
    
    
    book_dict = dict(zip(headers, [title,
                                   author,
                                   rating_value,
                                   rating_count,
                                   review_count,
                                   page,
                                   year]))
    
    return book_dict

In [186]:
len(overlap_book_url_list)

115

In [187]:
# Create list of info of the overlap books!
# (books that will be added)
overlap_book_info_list = []

for book_url in overlap_book_url_list:
    soup = get_soup(book_url)
    info = get_bookdata(soup)
    overlap_book_info_list.append(info)

In [188]:
len(overlap_book_info_list)

115

In [189]:
overlap_book_df = pd.DataFrame(overlap_book_info_list)

In [190]:
overlap_book_df

,book_title,author,rating_value,rating_count,review_count,page,year
0,Mary Poppins,P.L. Travers,4.02,117352,5563,209.0,1934
1,Bambi,Felix Salten,4.12,33168,770,192.0,1923
2,The Phantom of the Opera,Gaston Leroux,3.96,202793,7029,360.0,1909
3,The War of the Worlds,H.G. Wells,3.83,251010,8204,192.0,1898
4,Pride and Prejudice,Jane Austen,4.27,3099180,69778,279.0,1813
...,...,...,...,...,...,...,...
110,The Missing,Thomas Eidson,3.96,527,64,304.0,1995
111,The Witches,Peter Curtis,3.51,258,38,352.0,1960
112,The Private Lives of Pippa Lee,Rebecca Miller,3.22,2838,345,233.0,2002
113,The Handmaid's Tale,Margaret Atwood,4.12,1515549,72555,314.0,1985


In [193]:
# Save to pickle
overlap_book_df.to_pickle('../dump/book_data_add')

In [195]:
# Create list of author of ovarlap books
author_add_list = list(overlap_book_df.author)
len(author_add_list)

115

In [196]:
author_add_list_set = set(author_add_list)
len(author_add_list_set)

103

In [197]:
# Save the author_add_list
import pickle
with open('../dump/author_list_set_add', 'wb') as f:
    pickle.dump(author_add_list_set, f)

In [198]:
import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [199]:
%run -i '../py/fictiondb.py'

In [200]:
# Search on fictionDB for author to create autho_book_data_add
c = 0
author_book_add_list=[]
for author in author_add_list_set:
    c += 1
    print(c,author)
    result = search(author)
    author_book_add_list.append(result)

1 Tim Krabbé
can't find author
2 Ken Kesey
3 Michael Crichton
can't find author
4 Clair Huffaker
can't find author
5 Scott        Smith
6 Norman Maclean
can't find author
7 David Morrell
can't find author
8 George Orwell
can't find author
9 Thomas Eidson
can't find author
10 H.G. Wells
11 Joe Hill
12 John Steinbeck
can't find author
13 Henri Charrière
can't find author
14 Peter      Curtis
can't find author
15 Anthony Burgess
16 Leon Uris
can't find author
17 Karen Joy Fowler
can't find author
18 E.M. Forster
19 E. Nesbit
can't find author
20 Charlotte Brontë
can't find author
21 Bret Easton Ellis
can't find author
22 Patricia Highsmith
can't find author
23 Richard McKenna
24 Robert A. Heinlein
can't find author
25 Boris Pasternak
26 Paulo Coelho
can't find author
27 Oliver Sacks
can't find author
28 Henry Farrell
can't find author
29 Arthur Conan Doyle
can't find author
30 Gaston Leroux
31 Emily Brontë
can't find author
32 Don Winslow
can't find author
33 Thomas Berger
34 Pearl S. Buc

In [201]:
c = len(author_book_add_list)
c

103

In [202]:
# done_author=[]
# for dic in author_book_add_list:
#     for key, value in dic.items():
#         #print(key)
#         done_author.append(key)

# left_author_set = author_add_list_set.difference(set(done_author))
# len(done_author), len(left_author_set)

In [119]:
# %run -i '../py/fictiondb.py'
# for author in left_author_set:
#     c += 1
#     print(c,author)
#     result = search(author)
#     author_book_add_list.append(result)

In [204]:
# Create df for the author and year of publications
author_book_add_listfordf = []

for author in author_book_add_list:
    headers = ['author','years']
    for key, value in author.items():
        author = key
        years = value
    
    author_dict = dict(zip(headers,[author,years]))
    
    author_book_add_listfordf.append(author_dict)
    
author_book_df = pd.DataFrame(author_book_add_listfordf)
author_book_df

,author,years
0,Tim Krabbé,[]
1,Ken Kesey,"[2003, 1994, 1992, 1990, 1990, 1986, 1973, 196..."
2,Michael Crichton,"[2019, 2017, 2011, 2009, 2006, 2004, 2002, 199..."
3,Clair Huffaker,"[1979, 1979, 1979, 1978, 1978, 1978, 1977, 197..."
4,Scott Smith,"[2019, 2018, 2016, 2011, 2011, 2008, 2006, 1993]"
...,...,...
98,John A. Keel,"[2002, 1977]"
99,Donn Pearce,"[2005, 1986]"
100,Betty Smith,"[2012, 2000, 1987, 1971, 1966, 1947]"
101,Dashiell Hammett,"[2020, 2020, 2019, 2013, 2013, 2013, 2013, 201..."


In [205]:
# Save to pickle
import pickle
with open('../dump/author_book_data_add', 'wb') as f:
    pickle.dump(author_book_df, f)

In [215]:
# overlap_book_df

In [211]:
# Prepare to Google!
# Combine movie_df and book data
combined_df = pd.merge(movie_df,overlap_book_df,left_on='movie_title',right_on='book_title',how='inner')

In [217]:
combined_df

,movie_title,rating,vote,certificate,genre,release_date,metascore,keywords,budget,opening_weekend_usa,...,distributor,language,country,book_title,author,rating_value,rating_count,review_count,page,year
0,The Witches,5.3,21562,PG,"[Adventure, Comedy, Family]",2020-10-22,NaN,"[witch, european literature on screen, based o...",NaN,NaN,...,WarnerBros.,[English],"[USA, Mexico, UK]",The Witches,Peter Curtis,3.51,258,38,352.0,1960
1,The Witches,6.9,43886,PG,"[Adventure, Comedy, Family]",1990-08-24,78.0,"[witch, hotel, mouse, metamorphosis, child hater]",NaN,2221402.0,...,LorimarFilmEntertainment,[English],"[UK, USA]",The Witches,Peter Curtis,3.51,258,38,352.0,1960
2,Rebecca,6.0,28349,PG-13,"[Drama, Mystery, Romance]",2020-10-21,NaN,"[20th century literature on screen, widower, m...",NaN,NaN,...,Netflix,[English],"[UK, USA]",Rebecca,Daphne du Maurier,4.23,473032,25700,449.0,1938
3,Rebecca,8.1,123574,Approved,"[Drama, Mystery, Romance]",1940-04-12,86.0,"[widower, castle, shipwreck, costume party, ho...",1288000.0,NaN,...,SelznickInternationalPictures,"[English, French]",[USA],Rebecca,Daphne du Maurier,4.23,473032,25700,449.0,1938
4,Call Me by Your Name,7.9,210895,R,"[Drama, Romance]",2018-01-19,93.0,"[gay romance, male male kiss, father son relat...",4000000.0,412932.0,...,FrenesyFilmCompany,"[English, Italian, French, German, Hebrew]","[Italy, France, USA, Brazil]",Call Me by Your Name,André Aciman,4.22,232353,22868,248.0,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,Heidi,7.3,4108,Passed,"[Drama, Family, Musical]",1937-10-15,NaN,"[alps, female protagonist, footprint, horse dr...",NaN,NaN,...,TwentiethCenturyFox,[English],[USA],Heidi,Johanna Spyri,3.99,175008,4013,352.0,1880
125,The Good Earth,7.5,4492,Passed,"[Drama, Romance]",1937-08-06,NaN,"[locust swarm, farmer, locust, wealth, crop]",2800000.0,NaN,...,Metro-Goldwyn-Mayer(MGM),[English],[USA],The Good Earth,Pearl S. Buck,3.99,225865,10042,418.0,1931
126,The Laughing Policeman,6.4,2426,R,"[Crime, Drama, Thriller]",1974-01-23,NaN,"[mass murder, massacre, police procedural, bus...",2280000.0,NaN,...,TwentiethCenturyFox,[English],[USA],The Laughing Policeman,Maj Sjöwall,4.01,7799,590,224.0,1968
127,The Adventures of Tom Sawyer,7.1,1772,Passed,"[Adventure, Drama, Family]",1938-02-11,NaN,"[color remake of black and white film, sunday ...",NaN,NaN,...,SelznickInternationalPictures,[English],[USA],The Adventures of Tom Sawyer,Mark Twain,3.91,776997,10274,244.0,1876


In [242]:
essential_df = combined_df[['movie_title','author','genre']].copy()
essential_df['date'] = combined_df.release_date.dt.strftime('%Y-%m-%d')
essential_df.info()
essential_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 0 to 128
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_title  129 non-null    object
 1   author       129 non-null    object
 2   genre        129 non-null    object
 3   date         129 non-null    object
dtypes: object(4)
memory usage: 5.0+ KB


,movie_title,author,genre,date
0,The Witches,Peter Curtis,"[Adventure, Comedy, Family]",2020-10-22
1,The Witches,Peter Curtis,"[Adventure, Comedy, Family]",1990-08-24
2,Rebecca,Daphne du Maurier,"[Drama, Mystery, Romance]",2020-10-21
3,Rebecca,Daphne du Maurier,"[Drama, Mystery, Romance]",1940-04-12
4,Call Me by Your Name,André Aciman,"[Drama, Romance]",2018-01-19
...,...,...,...,...
124,Heidi,Johanna Spyri,"[Drama, Family, Musical]",1937-10-15
125,The Good Earth,Pearl S. Buck,"[Drama, Romance]",1937-08-06
126,The Laughing Policeman,Maj Sjöwall,"[Crime, Drama, Thriller]",1974-01-23
127,The Adventures of Tom Sawyer,Mark Twain,"[Adventure, Drama, Family]",1938-02-11


In [236]:
# Google search for book/author popularity
from get_book import *
%run -i '../py/fictiondb.py'
driver.get("https://www.google.com")

In [245]:
title_list = list(essential_df.movie_title)
author_list = list(essential_df.author)
date_list = list(essential_df.date)
genre_list = list(all_df.genre)
lng = len(title_list)
lng

129

In [246]:
book_history_2_add = []

In [247]:
for i in range(lng):
    book = title_list[i]
    author = author_list[i]
    date = date_list[i]
    genre = genre_list[i]
    
    print(i,book)
    
    info = book_google2(book,author,date,genre)
    book_history_2_add.append(info)

0 The Witches
1 The Witches
2 Rebecca
3 Rebecca
4 Call Me by Your Name
5 It's a Wonderful Life
6 One Flew Over the Cuckoo's Nest
7 A Clockwork Orange
8 Vertigo
9 Bird Box
10 Savages
11 Mary Poppins
12 The Town
13 Papillon
14 The Blue Lagoon
15 Black Beauty
16 Black Beauty
17 Cool Hand Luke
18 The Untouchables
19 First Blood
20 Chitty Chitty Bang Bang
21 Deliverance
22 The Maltese Falcon
23 Awakenings
24 Doctor Zhivago
25 Midnight Cowboy
26 Jane Eyre
27 Jane Eyre
28 Jane Eyre
29 The Hours
30 Fahrenheit 451
31 Dark Places
32 Treasure Island
33 Treasure Island
34 The Choice
35 Horns
36 The Thin Man
37 The Handmaid's Tale
38 A River Runs Through It
39 Suite Française
40 Howards End
41 Frankenstein
42 Frankenstein
43 The Ruins
44 Double Indemnity
45 Everything, Everything
46 Less Than Zero
47 The Jane Austen Book Club
48 Of Mice and Men
49 What Ever Happened to Baby Jane?
50 The Missing
51 Shane
52 Giant
53 Mysterious Skin
54 Strangers on a Train
55 East of Eden
56 Doctor Dolittle
57 Doctor

In [251]:
# Create df and pickle
book_history_2_add_df = pd.DataFrame(book_history_2_add)
book_history_2_add_df.to_pickle('../dump/book_history_2_data_add')

In [250]:
book_history_2_add_df

,title,release_date,genre,title_search,search_fiction_book,author_search,search_fiction_author,book_popularity,author_popularity
0,The Witches,2020-10-22,"{Crime, Drama}",188000.0,32600000.0,31100000.0,166000000,0.01,0.1873
1,The Witches,1990-08-24,"{Family, Fantasy, Adventure}",1890.0,5570.0,21400.0,79000,0.34,0.2709
2,Rebecca,2020-10-21,"{Romance, Drama}",96600.0,45400000.0,196000.0,189000000,0.00,0.0010
3,Rebecca,1940-04-12,"{Romance, Family, Drama}",1420000.0,56400000.0,2680000.0,1090000000,0.03,0.0025
4,Call Me by Your Name,2018-01-19,"{Action, Other, Adventure}",19700.0,38200000.0,64700.0,144000000,0.00,0.0004
...,...,...,...,...,...,...,...,...,...
124,Heidi,1937-10-15,"{Crime, Thriller, Drama}",924000.0,53000000.0,1200000.0,1050000000,0.02,0.0011
125,The Good Earth,1937-08-06,"{Romance, Drama}",7610000.0,111000000.0,26400000.0,1050000000,0.07,0.0251
126,The Laughing Policeman,1974-01-23,"{Romance, Comedy}",NaN,730.0,NaN,8450,NaN,NaN
127,The Adventures of Tom Sawyer,1938-02-11,"{Comedy, Crime, Drama}",2510000.0,38500000.0,61200000.0,1120000000,0.07,0.0546


In [4]:
import pandas as pd
# Combine book_history_2 and book_history_2_add
book_history_2_df = pd.read_pickle('../dump/book_history_2_data')
book_history_2_add_df = pd.read_pickle('../dump/book_history_2_data_add')
book_history_2_df.shape,book_history_2_add_df.shape

((1166, 9), (129, 9))

In [6]:
book_history_2_df_all = pd.concat([book_history_2_df,book_history_2_add_df])
book_history_2_df_all.to_pickle('../dump/book_history_2_data_all')
book_history_2_df_all.shape

(1295, 9)